In [2]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv("Dataset .csv")
df.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


In [4]:
df.columns

Index(['Restaurant ID', 'Restaurant Name', 'Country Code', 'City', 'Address',
       'Locality', 'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines',
       'Average Cost for two', 'Currency', 'Has Table booking',
       'Has Online delivery', 'Is delivering now', 'Switch to order menu',
       'Price range', 'Aggregate rating', 'Rating color', 'Rating text',
       'Votes'],
      dtype='object')

In [5]:
# Checking for missing values in the dataset
missing_values = df.isnull().sum()
missing_values

Restaurant ID           0
Restaurant Name         0
Country Code            0
City                    0
Address                 0
Locality                0
Locality Verbose        0
Longitude               0
Latitude                0
Cuisines                9
Average Cost for two    0
Currency                0
Has Table booking       0
Has Online delivery     0
Is delivering now       0
Switch to order menu    0
Price range             0
Aggregate rating        0
Rating color            0
Rating text             0
Votes                   0
dtype: int64

In [6]:
# Fill missing values in 'Cuisines' with a placeholder
df['Cuisines'].fillna('Unknown', inplace=True)

In [7]:
# Encode 'Cuisines' using one-hot encoding
cuisines_encoded = df['Cuisines'].str.get_dummies(sep=', ')


In [8]:
# Encode 'City' using one-hot encoding
city_encoded = pd.get_dummies(df['City'], prefix='City')

In [9]:
# Normalize the 'Price range' column
price_range_encoded = pd.get_dummies(df['Price range'], prefix='Price')


In [10]:
# Combine all encoded features into a single DataFrame
encoded_features = pd.concat([cuisines_encoded, city_encoded, price_range_encoded], axis=1)


In [11]:
# Add 'Aggregate rating' to the features for similarity calculations
encoded_features['Aggregate rating'] = df['Aggregate rating']

encoded_features.head()

,Afghani,African,American,Andhra,Arabian,Argentine,Armenian,Asian,Asian Fusion,Assamese,...,City_Weirton,City_Wellington City,City_Winchester Bay,City_Yorkton,City_��stanbul,Price_1,Price_2,Price_3,Price_4,Aggregate rating
0,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,True,False,4.8
1,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,True,False,4.5
2,0,0,0,0,0,0,0,1,0,0,...,False,False,False,False,False,False,False,False,True,4.4
3,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,True,4.9
4,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,True,4.8


In [12]:
# Define a sample user preference
user_preference = {
    'Cuisines': ['Japanese', 'Sushi'],
    'City': 'Makati City',
    'Price range': 3,
    'Aggregate rating': 4.5
}

In [13]:
# Encode user preferences
user_cuisines = pd.Series(0, index=cuisines_encoded.columns)
for cuisine in user_preference['Cuisines']:
    if cuisine in user_cuisines:
        user_cuisines[cuisine] = 1

In [14]:
user_city = pd.Series(0, index=city_encoded.columns)
user_city[f"City_{user_preference['City']}"] = 1


In [15]:
user_price = pd.Series(0, index=price_range_encoded.columns)
user_price[f"Price_{user_preference['Price range']}"] = 1

In [16]:
user_profile = pd.concat([user_cuisines, user_city, user_price])
user_profile['Aggregate rating'] = user_preference['Aggregate rating']

In [17]:
# Compute cosine similarity
similarity_scores = cosine_similarity([user_profile], encoded_features)[0]


In [18]:
# Get the top 10 most similar restaurants
top_indices = similarity_scores.argsort()[-10:][::-1]
recommended_restaurants = df.iloc[top_indices]

In [19]:
# Display the recommended restaurants
print(recommended_restaurants[['Restaurant Name', 'City', 'Cuisines', 'Price range', 'Aggregate rating']])

                   Restaurant Name         City                    Cuisines  \
1                 Izakaya Kikufuji  Makati City                    Japanese   
9287                    Sushi Masa      Jakarta             Sushi, Japanese   
9383                          Roka       London             Japanese, Sushi   
247                          Osaka    Davenport             Japanese, Sushi   
1319                    Sushi Haus      Gurgaon             Japanese, Sushi   
215       Fuji Japanese Steakhouse       Dalton             Japanese, Sushi   
549                         Sakura     Waterloo             Japanese, Sushi   
480   Fuji Bay Japanese Restaurant   Sioux City             Japanese, Sushi   
0                 Le Petit Souffle  Makati City  French, Japanese, Desserts   
1429                           IZU      Gurgaon             Japanese, Sushi   

      Price range  Aggregate rating  
1               3               4.5  
9287            3               4.9  
9383            